In [1]:
import numpy as np
import pandas as pd

In [2]:
# split to hour, minute, second

def add_time_string(df):
    df = df.reset_index(drop=True)
    df["queryts_new"] = pd.DataFrame(pd.to_datetime(df.queryts, unit='s'))
    df.queryts_new = df.queryts_new.astype(str)

    con = pd.DataFrame(df.queryts_new.str.split(" ",2).tolist(), columns = ['a','b'])
    con1 = pd.DataFrame(con.b.str.split(":",3).tolist(), columns = ['hr','mins','sec'])
    df["mins"] = con1.mins
    df.mins = pd.to_numeric(df.mins, errors = 'coerce')
    
    df = df.drop(
        ["customer_id","queryts","product_id","month","date","date_new","queryts_new"], axis=1)

    print (df.shape)
    return df

In [3]:
def count_interval_six(x):
    x = pd.Series(x)
    hr5 = sum(x < 15)
    hr11 = sum(x < 30)
    hr17 = sum(x < 45)
    hr23 = sum(x < 60)
    hr_0_5 = hr5
    hr_6_11 = hr11 - hr5
    hr_12_17 = hr17 - hr11
    hr_18_23 = hr23 - hr17
    return hr_0_5, hr_6_11, hr_12_17, hr_18_23

def groupby_count_interval_six(df):
    f = {'mins': lambda x: count_interval_six(x)}

    g = df.groupby(["file_id"]).aggregate(f).reset_index()
    g.columns = ["file_id", "count_interval_six_list"]

    g_ = g['count_interval_six_list'].apply(pd.Series)
    g_.columns = ["mins_0_14", "mins_15_29", "mins_30_44", "mins_45_59"]
    g = pd.concat([g[["file_id"]], g_], axis=1)
    
    return g

In [4]:
df = pd.DataFrame()
for file_ in ["train1","train2","train3","valid"]:
    df_ = pd.read_pickle("dataset/" + str(file_) + ".pkl")
    df = pd.concat([df, df_], axis=0)
    
df = add_time_string(df)
g_train = groupby_count_interval_six(df)

print (g_train.shape)

(54250245, 2)
(52518, 5)


In [5]:
df = pd.DataFrame()
for file_ in ["test_1","test_2","test_3"]:
    df_ = pd.read_pickle("dataset/" + str(file_) + ".pkl")
    df = pd.concat([df, df_], axis=0)

df = add_time_string(df)
g_test = groupby_count_interval_six(df)

print (g_test.shape)

(29022865, 2)
(29376, 5)


In [6]:
g = pd.concat([g_train, g_test], axis=0).reset_index(drop=True)

In [7]:
g.to_csv("dataset/count_min_interval_15.csv", index=False)